In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.anova import AnovaRM 

In [2]:
df = pd.read_csv("pre_data.csv", sep = ";")
key = pd.read_excel("cle.xlsx")
key.set_index("participant", inplace=True)


In [3]:
list_new_col = ([f"Trial{i}" for i in range(1, 13)])
key.columns = list_new_col

In [4]:
key.loc[1, "Trial1"]

'MO'

In [5]:
new_column_df = []
for i, column in enumerate(df.columns):
    nb_participant = column[1]
    nb_trial = column.split("_")[2]
    cdt = key.loc[int(nb_participant), nb_trial]
    if i%2 == 0:
        new_column_df.append(f"P{nb_participant}_X_{cdt}")
    else:
        new_column_df.append(f"P{nb_participant}_Y_{cdt}")
df.columns = new_column_df

In [6]:
df_try = df.groupby(by=df.columns, axis=1).mean()

C:\Users\alexl\AppData\Local\Temp\ipykernel_31048\1894595350.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_try = df.groupby(by=df.columns, axis=1).mean()


In [7]:
## Split les dataframes en fonction du participant 
list_df = []
for participant in range(1, 3):
    temp_df = df_try.filter(regex=f"P{participant}")
    list_df.append(temp_df)

In [11]:
dic_data = {}
df_column_clean = list(set([x[3:] for x in df_try.columns]))
final_df_mean = pd.DataFrame(columns=df_column_clean)
for df in list_df:
    par = df.columns[0][0:2]
    temp = pd.DataFrame.from_dict({col[3:]: [np.mean(df[col])] for col in df.columns})
    temp.index = [par]
    final_df_mean = pd.concat([final_df_mean, temp])

C:\Users\alexl\AppData\Local\Temp\ipykernel_32228\271851790.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df_mean = pd.concat([final_df_mean, temp])


In [13]:
final_df_mean.index.name = "participant"
final_df_mean.to_csv("data.csv", sep = ";")

In [ ]:
df[["axis_X, axis_Y"]] = df["condition"].str.split("_", expand=True)

In [21]:
model = AnovaRM(data = df, depvar='COP',
         subject="index", within=["condition"]).fit()
print(model)

                Anova
          F Value Num DF Den DF Pr > F
--------------------------------------
condition 29.9235 7.0000 7.0000 0.0001

